In [1]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive') 
#capire qua come fare prima di renderlo pubblico. 
#questa cella collega tutte le cartelle drive al notebook ma in realtà serve solo per accedere al file csv
#del democracy index

Mounted at /content/drive


In [3]:
req = r.get('https://ec.europa.eu/trustfundforafrica/navigation/all-projects_en') #pagina con tutti i progetti attivi dell'EUTF

In [ ]:
req

<Response [200]>

In [4]:
soup = BeautifulSoup(req.content,'html.parser')

In [5]:
elementi = soup.find_all(attrs={'class':'country__name'}) #trovo tutti gli elementi della pagina
                                                          #html che hanno come classe "country name"
nomi = list(map(lambda x: x.text,elementi)) #applico la funzione .text a ogni membro di elementi (così estraggo il testo come stringa)

In [6]:
nomi[0]
#serve levare a capi e spazi bianchi

'\n            Tchad          '

In [7]:
nomi = list(map(str.strip,nomi)) #voilà

In [8]:
nomi_unici = pd.Series(nomi).unique() #creiamo una serie di pandas dalla lista e togliamo i doppioni

In [9]:
nomi_unici = pd.Series(nomi_unici) #pandas però è stupido e la funzione unique() ritorna un array e non una serie, 
                                  #quindi ricreiamo una serie di pandas dall'array perché ci serviranno funzioni particolari

In [10]:
df = pd.read_csv('/content/drive/MyDrive/articoli in lavorazione/EUTF/EIU- materiale Notebook/global_oc_index_2021.csv',delimiter=';',decimal = ',')

In [11]:
df = df.dropna(how='all').dropna(axis=1,how='all') #colonne e righe vuote a caso, le eliminiamo

In [14]:
def valuta(x): 
  #funzione ausiliare che viene usata dopo. Le categorie le ho prese dallo stesso sito dell'Indice. 
    if x > 7.99:
        return 'Severe influence'
    elif x>5.99:
        return 'Significant influence'
    elif x>3.99:
        return 'Moderate influence'
    else:
        return 'Non-existent to little influence'

In [15]:
def ordina(indice):
  nindice=[]
  for x in indice: #la usiamo dopo, alla fine. Non è importante serve solo a ordinare i risultati
    if x == 'Severe influence':
      nindice.append(0)
    elif x=='Significant influence':
      nindice.append(1)
    elif x== 'Moderate influence':
      nindice.append(2)
    else:
      nindice.append(3)
  return nindice

In [19]:
def funzione(colonne:list, PRINT=False): #creiamo una mega funzione perché così è più comodo per il futuro
#se PRINT=True, viene anche stampato subito il risultato
    nuove_colonne=[]
    for colonna in colonne:
        df[f'categoria {colonna}'] = df[colonna].apply(valuta)
        nuove_colonne.append(f'categoria {colonna}')



    nomi_unici[~nomi_unici.isin(df['Country'])]



    #df[df['Region'].str.contains('Africa',flags=re.I)]['Country'].unique() 

    #questo comando qua sopra l'ho usato mentre stavo creando la funzione la prima volta e non sapevo 
    #quali Paesi andavano puliti, ora è inutile ovviamente quindi lo lascio come commento




    #qui manualmente abbiamo sistemato tutto
    nomi_unici[0]='Chad'
    nomi_unici[8]='Gambia'
    nomi_unici[13]='Mauritania'
    nomi_unici[20]='Guinea'
    nomi_unici[21] = 'Cameroon'




    nuovo_nomi = nomi_unici #solite cose per essere più chiari con i nomi delle variabili



    vecchi_nomi = pd.Series(nomi).unique() #solite cose per essere più chiari con i nomi delle variabili
    vecchi_nomi = pd.Series(vecchi_nomi) 



    diz = dict(zip(vecchi_nomi,nuovo_nomi)) #dizionario che usiamo per fare le sostituzioni vecchi_nomi --> nuovo_nomi


    nomi_pulito = list(map(lambda x: diz[x],nomi))


    vari_conti={} #per ogni colonna che selezioniamo nella funzione a monte, 
    #questo dizionario sarà così: {nome colonna : lista}, in cui la lista contiene 
    #per ogni progetto di EUTF la categoria (da severe a non-existent) legata all'indice selezionato 
    #nelle varie colonne

    ndf = df.set_index('Country')

    for country in nomi_pulito:
        if country in ndf.index:
            row = ndf.loc[country]
            for colonna in nuove_colonne:
                vari_conti[colonna] = vari_conti.get(colonna,[]) + [row[colonna]]        
        else:
            vari_conti[colonna] = vari_conti.get(colonna,[]) + ['NC']

    risultato = {}
    for colonna,lista in vari_conti.items():
        lista_finale = pd.Series(lista).value_counts()
        lista_finale = lista_finale.sort_index(key=ordina)

        if PRINT:
          print(f'\n{colonna}\n\n',lista_finale,'\n-----------\n',sep='')

        risultato[colonna] = lista_finale
    return risultato

In [20]:
finale = funzione(['CRIMINAL MARKETS','State-embedded actors','Mafia-style groups'],PRINT=True )


categoria CRIMINAL MARKETS

Significant influence                46
Moderate influence                  137
Non-existent to little influence      3
dtype: int64
-----------


categoria State-embedded actors

Severe influence         105
Significant influence     55
Moderate influence        26
dtype: int64
-----------


categoria Mafia-style groups

Severe influence                    21
Significant influence               32
Moderate influence                  51
Non-existent to little influence    82
dtype: int64
-----------



In [35]:
finale['categoria State-embedded actors']

Severe influence         105
Significant influence     55
Moderate influence        26
dtype: int64

In [29]:
dir(finale['categoria State-embedded actors'])

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__r